# LangChain链的基本使用

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

## 使用管道操作符来实现一条链

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("讲一个关于{topic}的笑话，不要有任何解释")

chain = prompt | llm | StrOutputParser()
response = chain.invoke({"topic": "狗"})
print(response)

一只狗走进酒吧，对酒保说：“我能不能要一杯……呃……一杯……”  
酒保问：“怎么了？舌头被咬住了？”  
狗：“不，我是金毛，天生就这口音！”


In [9]:
# 流式输出

for chunk in chain.stream({"topic": "猫"}):
    print(chunk, end="", flush=True)

有一天，一只猫走进酒吧，跳到吧台上对酒保说：“给我来一杯牛奶。”  
酒保惊讶地问：“哇！会说话的猫？你从哪儿学的这本事？”  
猫淡定地舔舔爪子：“现在才问？牛奶都摆这儿好几年了。”

In [10]:
# 异步流式输出
async for chunk in chain.astream({"topic": "猫"}):
    print(chunk, end="", flush=True)

有一天，一只猫走进图书馆，走到前台对管理员说：“喵。”  

管理员点点头，递给它一本《如何驯服人类》。  

猫看了一眼书名，把书推回去：“喵。”（翻译：换一本。）  

管理员又递上《如何让你的猫更爱你》。  

猫满意地叼起书走了，心想：“终于轮到我来研究了。”

In [ ]:
# 打印有哪些 事件流
events = []
async for event in chain.astream_events({"topic": "猫"}, version="v2"):
    events.append(event)
    print(event, end="\n", flush=True)

{'event': 'on_chain_start', 'data': {'input': {'topic': '猫'}}, 'name': 'RunnableSequence', 'tags': [], 'run_id': 'b7c0ce83-30cc-4bd8-989f-81c86af8cba7', 'metadata': {}, 'parent_ids': []}
{'event': 'on_prompt_start', 'data': {'input': {'topic': '猫'}}, 'name': 'ChatPromptTemplate', 'tags': ['seq:step:1'], 'run_id': '2a87e20f-c49c-4db2-b918-c77e9a24ba2d', 'metadata': {}, 'parent_ids': ['b7c0ce83-30cc-4bd8-989f-81c86af8cba7']}
{'event': 'on_prompt_end', 'data': {'output': ChatPromptValue(messages=[HumanMessage(content='讲一个关于猫的笑话，不要有任何解释', additional_kwargs={}, response_metadata={})]), 'input': {'topic': '猫'}}, 'run_id': '2a87e20f-c49c-4db2-b918-c77e9a24ba2d', 'name': 'ChatPromptTemplate', 'tags': ['seq:step:1'], 'metadata': {}, 'parent_ids': ['b7c0ce83-30cc-4bd8-989f-81c86af8cba7']}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='讲一个关于猫的笑话，不要有任何解释', additional_kwargs={}, response_metadata={})]]}}, 'name': 'ChatOpenAI', 'tags': ['seq:step:2'], 'run_id

In [14]:
# 事件过滤-按name
# 按照运行时配置的name来过滤事件
from langchain_core.output_parsers import JsonOutputParser

chain = llm.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format."
    "Use a dict with an outer key of 'countries' which contains a list of countries."
    "Each country should be a dict with keys 'name' and 'population'.",
    include_names=["my_parser"],
    version="v2",
):
    print(event)
    max_events += 1
    if max_events > 10:
        print("...")  # indicate there are more events
        break

{'event': 'on_parser_start', 'data': {'input': "output a list of the countries france, spain and japan and their populations in JSON format.Use a dict with an outer key of 'countries' which contains a list of countries.Each country should be a dict with keys 'name' and 'population'."}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': '34d1df31-6d6d-460b-af00-2b5273ad7294', 'metadata': {}, 'parent_ids': ['a44341c8-bff3-47fa-9bd2-0f4f9eca73a9']}
{'event': 'on_parser_stream', 'run_id': '34d1df31-6d6d-460b-af00-2b5273ad7294', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {}}, 'parent_ids': ['a44341c8-bff3-47fa-9bd2-0f4f9eca73a9']}
{'event': 'on_parser_stream', 'run_id': '34d1df31-6d6d-460b-af00-2b5273ad7294', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': []}}, 'parent_ids': ['a44341c8-bff3-47fa-9bd2-0f4f9eca73a9']}
{'event': 'on_parser_stream', 'run_id': '34d1df31-6d6d-460b-af00-2b5273ad7294', 'name': 'my

In [16]:
# 事件过滤-按tag标签
from langchain_core.output_parsers import JsonOutputParser

chain = llm | JsonOutputParser().with_config({"tags": ["my_chain"]})

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format."
    "Use a dict with an outer key of 'countries' which contains a list of countries."
    "Each country should be a dict with keys 'name' and 'population'.",
    include_names=["my_chain"],
    version="v2",
):
    print(event)
    max_events += 1
    if max_events > 10:
        print("...")  # indicate there are more events
        break

In [18]:
# 事件阶段过滤
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format."
    "Use a dict with an outer key of 'countries' which contains a list of countries."
    "Each country should be a dict with keys 'name' and 'population'.",
    version="v2",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(
            f"Parser chunk: {event['data']['chunk']}",
            flush=True,
        )
    max_events += 1
    if max_events > 30:
        print("...")  # indicate there are more events
        break

Chat model chunk: 'Here'
Chat model chunk: ' is'
Chat model chunk: ' the'
Chat model chunk: ' JSON'
Chat model chunk: ' output'
Chat model chunk: ' with'
Chat model chunk: ' the'
Chat model chunk: ' requested'
Chat model chunk: ' structure'
Chat model chunk: ':\n\n'
Chat model chunk: '```'
Chat model chunk: 'json'
Chat model chunk: '\n'
Chat model chunk: '{\n'
...


## 使用pipe方法来实现一条链

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("讲一个关于{topic}的笑话，不要有任何解释")

chain = prompt.pipe(llm).pipe(StrOutputParser())
response = chain.invoke({"topic": "狗"})
print(response)

一只狗走进酒吧，对酒保说：“我能不能要一杯啤酒……再听听今天的特价菜？”

酒保震惊地说：“天啊！会说话的狗！这简直太不可思议了！”

狗翻了个白眼：“哇哦，会说话的狗才不可思议？你刚才可是毫不犹豫地准备给我倒啤酒呢。”


## 链的并行执行

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

# 定义两个链
joke_chain = (
    ChatPromptTemplate.from_template("讲一个关于{topic}的笑话，不要有任何解释") | llm
)
poem_chain = ChatPromptTemplate.from_template("写一首关于{topic}的诗") | llm

# 合并两个链
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

# 执行链
map_chain.invoke({"topic": "猫"})

{'joke': AIMessage(content='为什么猫总是输掉扑克比赛？  \n\n因为它们的“扑克脸”太明显了——每次抓到好牌，尾巴就翘上天了！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 13, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'deepseek-v3-250324', 'system_fingerprint': None, 'id': '02175755978315664ec4d8b76aec9b104893191049097998cf469', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1db8f91-01a7-4b6c-b67d-f19985f7c9ec-0', usage_metadata={'input_tokens': 13, 'output_tokens': 32, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 'poem': AIMessage(content='《猫的十四行》\n\n\n它把整个下午蜷成毛线球，\n在窗台边，在光与影的间隙。\n偶尔伸出前爪试探风的去向，\n却始终守着那寸温暖的领地。\n\n胡须丈量过所有黑暗的角落，\n瞳孔里竖着整座夜的水晶。\n当它跃起时，空气泛起涟漪，\n像一篇突然押

In [22]:
! pip install grandalf

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/05/e7/df2285f3d08fee213f2d041540fa4fc9ca6c2d44cf36d3a035bf2a8d2bcc/pyparsing-3.2.3-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [grandalf]


In [ ]:
# 查看链的图形表示
# map_chain.get_graph()
map_chain.get_graph().print_ascii()

               +--------------------------+                
               | Parallel<joke,poem>Input |                
               +--------------------------+                
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+*                     +------------+     
                   ***               ***                   
                      ***         ***   